# Overview

Normally we use sentence-transformers model as the default encoder in ML project. However, as the recently research LLM2Vec can convert decoder-only LLMs into text encoders with three steps. So, let's see how to use it.

In [1]:
!pip install -U -q llm2vec==0.1.4

In [2]:
import os

os.environ["model_name"]="McGill-NLP/LLM2Vec-Sheared-LLaMA-mntp"

In [3]:
import torch
from llm2vec import LLM2Vec
from transformers import AutoTokenizer, AutoModel, AutoConfig
from peft import PeftModel


tokenizer=AutoTokenizer.from_pretrained(os.getenv('model_name'))

tokenizer_config.json:   0%|          | 0.00/871 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/434 [00:00<?, ?B/s]

In [4]:
config=AutoConfig.from_pretrained(os.getenv('model_name'), trust_remote_code=True)

config.json:   0%|          | 0.00/753 [00:00<?, ?B/s]

In [5]:
model=AutoModel.from_pretrained(os.getenv('model_name'), trust_remote_code=True, config=config, torch_dtype=torch.bfloat16, device_map="cuda")

adapter_config.json:   0%|          | 0.00/791 [00:00<?, ?B/s]

modeling_llama_encoder.py:   0%|          | 0.00/8.21k [00:00<?, ?B/s]

attn_mask_utils.py:   0%|          | 0.00/10.1k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/McGill-NLP/LLM2Vec-Sheared-LLaMA-mntp:
- attn_mask_utils.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/McGill-NLP/LLM2Vec-Sheared-LLaMA-mntp:
- modeling_llama_encoder.py
- attn_mask_utils.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


pytorch_model.bin:   0%|          | 0.00/5.38G [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of the model checkpoint at princeton-nlp/Sheared-LLaMA-1.3B were not used when initializing LlamaEncoderModel: ['lm_head.weight']
- This IS expected if you are initializing LlamaEncoderModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LlamaEncoderModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification mode

adapter_model.safetensors:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

In [6]:
model=PeftModel.from_pretrained(
    model,
    os.getenv('model_name')
)

In [7]:
# Wrapper for encoding and pooling operations
l2v=LLM2Vec(model, tokenizer, pooling_mode="mean", max_length=512)

# Encoding Queries

In [12]:
instruction=("Given a web search query, retrieve relevant passagers that answer the query:")

queries=[
    [instruction, "How the weather today in Melbourne"],
    [instruction, "Melbourne Parkville suburb"],
]

q_reps=l2v.encode(queries)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

TypeError: AttentionMaskConverter._unmask_unattended() got an unexpected keyword argument 'unmasked_value'

# Encoding Documents

In [ ]:
documents=[
    """Melbourne is notorious for its fickle weather, often experiencing "four seasons in one day." Summers (December to February) are warm to hot, with average temperatures ranging from 14°C to 25°C, but occasional heatwaves can push the mercury above 40°C. Autumn (March to May) brings milder temperatures and stunning foliage, while winters (June to August) are cool and wet, with average temperatures between 6°C and 14°C. Spring (September to November) is a delightful time with blossoming flowers and temperatures gradually warming up. It's always wise to be prepared for anything when heading outdoors in Melbourne!"""
]

d_reps=l2v.encode(documents)

In [ ]:
q_res_norm=torch.nn.functional.normalize(q_res, p=2, dim=1)
d_res_norm=torch.nn.functional.normalize(d_res, p=2, dim=1)
cos_sim=torch.mm(q_reps_norm, d_reps_norm.transpose(0,1))
cos_sim